# 🧴 Skin Condition Classifier Training

**Categories:** acne, blackheads, clear_skin, dark_spots, puffy_eyes, wrinkles

## Setup
1. Upload `training.zip` ke Colab (drag & drop ke sidebar kiri)
2. Jalankan semua cell dari atas ke bawah
3. Download hasil model `skin-model.zip`

In [ ]:
# Install dependencies
!pip install tensorflowjs -q

## 📁 Step 1: Upload & Extract Dataset
1. Di laptop, zip folder `data/training/` menjadi `training.zip`
2. Upload `training.zip` ke sidebar kiri Colab (drag & drop)
3. Jalankan cell di bawah

In [ ]:
# Extract training.zip
!unzip -q /content/training.zip -d /content/

# Cek isi folder
print("Folder contents:")
!ls /content/training/

In [ ]:
import os
import json
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflowjs as tfjs
import numpy as np

print(f"TensorFlow: {tf.__version__}")
print(f"GPU: {tf.config.list_physical_devices('GPU')}")

In [ ]:
# =============================================================================
# CONFIGURATION
# =============================================================================

DATASET_PATH = '/content/training'

CONFIG = {
    'image_size': 128,
    'batch_size': 32,
    'epochs': 15,
    'learning_rate': 0.001,
    'validation_split': 0.2,
}

LABELS = ['acne', 'blackheads', 'clear_skin', 'dark_spots', 'puffy_eyes', 'wrinkles']

FOLDER_MAP = {
    'acne': 'acne',
    'blackheads': 'blackheads',
    'clear skin': 'clear_skin',
    'clear_skin': 'clear_skin',
    'dark spots': 'dark_spots',
    'dark_spots': 'dark_spots',
    'puffy eyes': 'puffy_eyes',
    'puffy_eyes': 'puffy_eyes',
    'wrinkles': 'wrinkles',
}

In [ ]:
# =============================================================================
# LOAD DATASET
# =============================================================================

def load_dataset():
    images = []
    labels_list = []
    
    print("Loading images...")
    for folder in os.listdir(DATASET_PATH):
        folder_path = os.path.join(DATASET_PATH, folder)
        if not os.path.isdir(folder_path):
            continue
        
        label = FOLDER_MAP.get(folder.lower())
        if label is None or label not in LABELS:
            print(f"  Skip folder: {folder}")
            continue
        
        label_idx = LABELS.index(label)
        count = 0
        
        for filename in os.listdir(folder_path):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.webp')):
                img_path = os.path.join(folder_path, filename)
                try:
                    img = keras.preprocessing.image.load_img(
                        img_path, 
                        target_size=(CONFIG['image_size'], CONFIG['image_size'])
                    )
                    img_array = keras.preprocessing.image.img_to_array(img)
                    img_array = (img_array / 127.5) - 1
                    images.append(img_array)
                    labels_list.append(label_idx)
                    count += 1
                except Exception as e:
                    pass
        
        print(f"  {label}: {count} images")
    
    return np.array(images), np.array(labels_list)

X, y = load_dataset()
print(f"\nTotal: {len(X)} images")

In [ ]:
# =============================================================================
# BUILD MODEL
# =============================================================================

def build_model():
    model = keras.Sequential([
        layers.Input(shape=(CONFIG['image_size'], CONFIG['image_size'], 3)),
        
        # Block 1
        layers.Conv2D(32, 3, padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2),
        
        # Block 2
        layers.Conv2D(64, 3, padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2),
        
        # Block 3
        layers.Conv2D(128, 3, padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.GlobalAveragePooling2D(),
        
        # Classifier
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(len(LABELS), activation='softmax'),
    ])
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=CONFIG['learning_rate']),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

model = build_model()
model.summary()

In [ ]:
# =============================================================================
# TRAIN
# =============================================================================

# Shuffle
indices = np.random.permutation(len(X))
X, y = X[indices], y[indices]

# Split
split = int(len(X) * (1 - CONFIG['validation_split']))
X_train, X_val = X[:split], X[split:]
y_train, y_val = y[:split], y[split:]

print(f"Train: {len(X_train)}, Val: {len(X_val)}")

# Train!
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=CONFIG['epochs'],
    batch_size=CONFIG['batch_size'],
)

In [ ]:
# =============================================================================
# SAVE MODEL
# =============================================================================

OUTPUT_PATH = '/content/output'
os.makedirs(OUTPUT_PATH, exist_ok=True)

# Export ke TensorFlow.js format
tfjs_path = os.path.join(OUTPUT_PATH, 'tfjs')
tfjs.converters.save_keras_model(model, tfjs_path)
print(f"Model saved to: {tfjs_path}")

# Save labels
with open(os.path.join(OUTPUT_PATH, 'labels.json'), 'w') as f:
    json.dump(LABELS, f)
print(f"Labels saved")

# List output files
print("\nOutput files:")
for root, dirs, files in os.walk(OUTPUT_PATH):
    for f in files:
        print(f"  {os.path.join(root, f)}")

In [ ]:
# =============================================================================
# DOWNLOAD MODEL
# =============================================================================

!cd /content && zip -r skin-model.zip output/

from google.colab import files
files.download('/content/skin-model.zip')

print("\nDownload selesai!")
print("Extract zip dan copy isi folder 'output' ke: public/models/skin-classifier/")